## Enviorment
The following section are used to specify the hyperparameter and the path the the software that were used. 
No need to changed if the pipeline stay the same and were applied to different data


In [60]:
##!/bin/sh
# MAKE SURE FUSION.compute_weights.R IS IN YOUR PATH
# FILL IN THESE PATHS
# For mac user, the mac version of GCTA shall be downloaded saperately, the one came with the Fusion package will not work.
GCTA="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/gcta_1.93.2beta_mac/gcta64"
PLINK="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/plink_mac_20200616/plink"
GEMMA="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/GEMMA"
PLINKED_file="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/genotype_plink_bfiles"

# Required the customized fusion.compute_weight.mod.R script, other wise will not work
Rscp="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/FUSION.compute_weights.mod.R"
# Path to the input data
Datarep="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/"
# ALTERNATIVELY: ENSURE THAT plink, gcta, gemma CAN BE CALLED FROM PATH AND REMOVE --PATH_* FLAGS BELOW

# PATH TO WORKING DIRECTORY
wd="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/working"

# PATH TO OUTPUT DIRECTORY (population-specific subdirs will be made)
OUT_DIR="./WEIGHTS"

# ROWS IN THE MATRIX TO ANALYZE (FOR BATCHED RUNS)
BATCH_START=1
BATCH_END=49

## Input 

This is where the actual RNA Seq data of our own should be used as input.

Note: GD462.GeneQuantRPKM.50FN.samplename.resk10.txt is the rna seq data for gene expression from the "465 lymphoblastoid cell lines from the 1000 Genomes" projects. There is 23722 genes from 462 samples. The first 4 col is "TargetID, Gene_Symbol, Chr, Coord". The rest are each samples.

GD462.GeneQuantRPKM.50FN.samplename.resk10.txt is the RNA seq data for the first 10 ppl

LDREF/1000G.EUR.*.bim contains the linkage disequilibrium info for each snp.

In [50]:
# PATH TO DIRECTORY CONTAINING LDREF DATA (FROM FUSION WEBSITE or https://data.broadinstitute.org/alkesgroup/FUSION/LDREF.tar.bz2)
LDREF="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/genotype_plink_bfiles"
# THIS IS USED TO RESTRICT INPUT SNPS TO REFERENCE IDS ONLY

# PATH TO GEUVADIS GENE EXPRESSION MATRIX:
PRE_GEXP="fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt"
# GEUVADIS DATA WAS DOWNLOADED FROM https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/files/analysis_results/

# PATH TO PREFIX FOR GEUVADIS GENOTYPES SPLIT BY CHROMOSOME
# SUBSAMPLE THESE TO THE LDREF SNPS FOR EFFICIENCY
PRE_GENO="YRI.120samples.hg19"
echo $PRE_GENO

YRI.120samples.hg19



## Pipeline


In [51]:
# --- BEGIN SCRIPT:
cd $wd
NR="${BATCH_START}_${BATCH_END}"
mkdir -p tmp/$NR
mkdir -p hsq/$NR
mkdir -p out/$NR
# THIS IS DIRECTORY WHERE THE OUTPUT WILL GO:
mkdir $OUT_DIR

Note 1: the first two column of the out.pheno shall be the same as that of the .fam file

In [56]:
# makes necessary component that are useful for follow up pipeline
cd $wd

# get the within family sample ID, need to customized for it to match those of the LDREF file
head -1 $Datarep$PRE_GEXP | awk '{$1=$2=$3=$4=""; print substr($0,5)}' | fmt -1 | awk '$0="NA"$0' > PRE_GEXPID
# make a diretory containing the per_gene file
mkdir $wd/${PRE_GEXP%.*}_per_gene


## SOS Pipeline prep (WIP)

In [57]:
cd $wd

# extract each gene from the data matrixs to a file
cat $Datarep$PRE_GEXP | awk -v s=$BATCH_START -v e=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do

GNAME=`echo $PARAM | awk '{ print $4 }'`
echo $GNAME
touch $wd/${PRE_GEXP%.*}_per_gene/${PRE_GEXP%.*}_$GNAME.txt
echo $PARAM > $wd/${PRE_GEXP%.*}_per_gene/${PRE_GEXP%.*}_$GNAME.txt

done

ENSG00000225972.1
ENSG00000225630.1
ENSG00000237973.1
ENSG00000229344.1
ENSG00000248527.1
ENSG00000198744.5
ENSG00000187634.6
ENSG00000188976.6
ENSG00000187961.9
ENSG00000187583.6
ENSG00000188290.6
ENSG00000187608.5
ENSG00000188157.9
ENSG00000242590.1
ENSG00000217801.5
ENSG00000273443.1
ENSG00000131591.13
ENSG00000186891.9
ENSG00000186827.6
ENSG00000078808.12
ENSG00000176022.3
ENSG00000160087.16
ENSG00000131584.14
ENSG00000169972.7
ENSG00000127054.14
ENSG00000240731.1
ENSG00000224051.2
ENSG00000107404.13
ENSG00000175756.9
ENSG00000221978.7
ENSG00000224870.3
ENSG00000242485.1
ENSG00000264293.1
ENSG00000272455.1
ENSG00000179403.10
ENSG00000160072.15
ENSG00000197785.9
ENSG00000160075.10
ENSG00000215791.2
ENSG00000228594.1
ENSG00000272106.1
ENSG00000197530.8
ENSG00000248333.3
ENSG00000272004.1
ENSG00000189339.7
ENSG00000269737.1
ENSG00000269227.1
ENSG00000008128.18
ENSG00000268575.1



## Original Pipeline (Working but needs to transformed into sos pipeline)

In [59]:
cd $wd
# Loop through each gene expression phenotype in the batch (Only looking at the first 10 genes)
cat $Datarep$PRE_GEXP | awk -v s=$BATCH_START -v e=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do
# Get the gene positions +/- 500kb
CHR=`echo $PARAM | awk '{ print $1 }'`
P0=`echo $PARAM | awk '{ print $2 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $2 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $4 }'`

OUT="tmp/$NR/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1

## Pull out the current gene expression phenotype (Note 1 )
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste PRE_GEXPID PRE_GEXPID - > $OUT.pheno
## Get the locus genotypes for all samples and set current gene expression as the phenotype
$PLINK --bfile $LDREF/$PRE_GENO.$CHR --pheno $tmpdir$OUT.pheno --make-bed --out $tmpdir$OUT --chr $CHR --from-bp $P0 --to-bp $P1 --extract $LDREF/$PRE_GENO.$CHR.bim --keep $tmpdir$OUT.pheno --allow-no-sex
## Process all samples together (for reference purposes only since this is mult-ethnic data)
mkdir $OUT_DIR/ALL
FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"
#echo $FINAL_OUT
#
#echo $OUT.tmp

Rscript $Rscp --bfile $OUT --tmp $OUT.tmp --out $FINAL_OUT --verbose 0 --save_hsq --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,lasso,top1,enet

#
## Append heritability output to hsq file
cat $FINAL_OUT.hsq >> hsq/$NR.hsq
#
## Clean-up just in case
rm -f $FINAL_OUT.hsq $OUT.tmp.*
#
# Remove all intermediate files
echo "end of circle"

# GO TO THE NEXT GENE
done




ENSG00000225972.1 chr1 64442 1064442
PLINK v1.90b6.18 64-bit (16 Jun 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_100/fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt.ENSG00000225972.1.log.
Options in effect:
  --allow-no-sex
  --bfile /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/genotype_plink_bfiles/YRI.120samples.hg19.chr1
  --chr chr1
  --extract /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/chicago/Data/genotype_plink_bfiles/YRI.120samples.hg19.chr1.bim
  --from-bp 64442
  --keep tmp/1_100/fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt.ENSG00000225972.1.pheno
  --make-bed
  --out tmp/1_100/fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt.ENSG00000225972.1
  --pheno tmp/1_100/fastqtl_qqnorm_RNAseqGeuvadis_phase2.txt.ENSG00000225972.1.pheno
  --to-bp 1064442

16384 MB RAM detected; reserving 8192 MB for main workspace.
3399

In [15]:
wget https://github.com/cumc/neuro-twas/blob/master/WIP/pipeline_prototype/Fusion.compute_weights_testing.ipynb

--2020-07-13 14:31:25--  https://github.com/cumc/neuro-twas/blob/master/WIP/pipeline_prototype/Fusion.compute_weights_testing.ipynb
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-07-13 14:31:26 ERROR 404: Not Found.




In [120]:
cd tmp

In [73]:
cd tmp/1_2/
ls
cat GD462.hsq_succ.test.txt.ENSG00000136237.12..tmp.cv.log

'utf-8' codec can't decode byte 0xa4 in position 1: invalid start byte


In [74]:
rm -r *

'utf-8' codec can't decode byte 0x8c in position 3: invalid start byte


In [72]:
cat 1_10.hsq | grep . -c

cat: 1_10.hsq: No such file or directory
0



In [26]:
echo $pop

In [123]:
head /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.10.bed

head: /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.10.bed: No such file or directory



In [48]:
ls WEIGHTS/*/*

WEIGHTS/ALL/ALL.ENSG00000127054.14.wgt.RDat
WEIGHTS/ALL/ALL.ENSG00000169972.7.wgt.RDat
WEIGHTS/ALL/ALL.ENSG00000224051.2.wgt.RDat
WEIGHTS/ALL/ALL.ENSG00000272455.1.wgt.RDat



patential working genes
ENSG00000225267.1
ENSG00000259382.1
ENSG00000135046.9

In [25]:
$OUT

In [24]:
head -1 $Datarep$PRE_GEXP

TargetID	Gene_Symbol	Chr	Coord	HG00096	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103	HG00104	HG00105	HG00106	HG00108	HG00109	HG00110	HG00111	HG00112	HG00114	HG00115	HG00116	HG00117	HG00118	HG00119	HG00120	HG00121	HG00122	HG00123	HG00124	HG00125	HG00126	HG00127	HG00128	HG00129	HG00130	HG00131	HG00132	HG00133	HG00134	HG00135	HG00136	HG00137	HG00138	HG00139	HG00141	HG00142	HG00143	HG00145	HG00146	HG00148	HG00149	HG00150	HG00151	HG00152	HG00154	HG00155	HG00156	HG00157	HG00158	HG00159	HG00160	HG00171	HG00173	HG00174	HG00176	HG00177	HG00178	HG00179	HG00180	HG00181	HG00182	HG00183	HG00185	HG00186	HG00187	HG00188	HG00189	HG00231	HG00232	HG00233	HG00234	HG00235	HG00236	HG00238	HG00239	HG00240	HG00242	HG00243	HG00244	HG00245	HG00246	HG00247	HG00249	HG00250	HG00251	HG00252	HG00253	HG00255	HG00256	HG00257	HG00258	HG00259	HG00260	HG00261	HG00262	HG00263	HG00264	HG00265	HG00266	HG00267	HG00268	HG00269	HG00271	HG00272	HG00273	HG00274	HG00275	HG00276	HG00277	HG00278	HG00280	HG00281	HG00282	HG00284	H

# Association Test
The following section is for the assoc test, which is ready to use off the shelf, tne triky part is getting the weights

In [91]:
Rscr2="<>"
# This is the new input file
sumstats="<>"
weights=FINAL_OUT
weights_dir=$wd/WEIGHTS/
ref_ld_chr=$LDREF/PRE_GENO
chr="<>"


In [ ]:
Rscript $Rscr2 \
--sumstats $sumstats \
--weights $weights \
--weights_dir $weights_dir \
--ref_ld_chr $LDREF/PRE_GENO \
--chr 22 \
--out PGC2.SCZ.22.dat